<a href="https://colab.research.google.com/github/4L3M4R/cerbero/blob/main/cerbero-pre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===============================================
#             CERBERO PREMARKET
#   Descarga noticias y calcula sentiment
#   (Actualiza el día anterior en los archivos *_datos.txt)
# ===============================================

import os
import pandas as pd
import feedparser
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from datetime import datetime, timedelta
from urllib.parse import quote

# ===============================================
#             CONFIGURACIÓN
# ===============================================

# Lista de activos
activos = {}
with open("activos.txt", "r") as f:
    for line in f:
        symbol, source, search_name = line.strip().split(":")
        activos[symbol.strip()] = {
            "source": source.strip().lower(),
            "search_name": search_name.strip()
        }

# Inicialización de analizadores de sentimiento
nltk.download('vader_lexicon')
vader_analyzer = SentimentIntensityAnalyzer()
finbert_tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
finbert_model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")

# ===============================================
#           FUNCIONES AUXILIARES
# ===============================================

def registrar_log(message, log_file="run_summary_pre.log"):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(log_file, "a") as log:
        log.write(f"[{timestamp}] {message}\n")

def descargar_noticias_y_calcular_sentiment(symbol, search_name):
    os.makedirs("logs", exist_ok=True)

    query = quote(search_name)
    feed = feedparser.parse(f"https://news.google.com/rss/search?q={query}")
    noticias = []
    for entry in feed.entries:
        noticias.append({
            "timestamp": entry.published,
            "title": entry.title,
            "link": entry.link
        })

    df = pd.DataFrame(noticias)
    if df.empty:
        registrar_log(f"{symbol} - No se encontraron noticias")
        return

    # Calcular sentimiento con VADER (y placeholder para FinBERT)
    df["vader_sentiment"] = df["title"].apply(lambda x: vader_analyzer.polarity_scores(x)["compound"])
    df["finbert_sentiment"] = 0  # Puedes reemplazar con análisis FinBERT si lo deseas

    # Calcular promedios
    vader_promedio = df["vader_sentiment"].mean()
    finbert_promedio = df["finbert_sentiment"].mean()

    # Actualizar el archivo de datos
    actualizar_sentimiento(symbol, vader_promedio, finbert_promedio)

    # Guardar noticias procesadas
    today_str = pd.Timestamp.utcnow().date()
    df.to_csv(f"logs/{symbol}_nuevas_agregadas_{today_str}.csv", index=False)
    registrar_log(f"{symbol} - Guardadas {len(df)} noticias con sentiment")

def actualizar_sentimiento(symbol, vader_promedio, finbert_promedio):
    """
    Actualiza el archivo *_datos.txt del símbolo con el sentiment promedio calculado.
    Se integra con el flujo del postmarket, respetando formato tab-separado.
    En premarket, actualiza la última fila (previa).
    """
    import os
    import pandas as pd
    from datetime import datetime, timedelta

    filename = f"{symbol}_datos.txt"

    # --- Cargar archivo existente ---
    if not os.path.exists(filename):
        print(f"⚠️ Archivo no encontrado: {filename}. Creando nuevo archivo.")
        # Crear DataFrame mínimo
        df = pd.DataFrame([{
            "timestamp": (datetime.utcnow().date() - timedelta(days=1)).strftime("%Y-%m-%d"),
            "vader": vader_promedio,
            "finbert": finbert_promedio
        }])
        df.to_csv(filename, sep="\t", index=False)
        return

    df = pd.read_csv(filename, sep="\t")
    print(f"✅ Archivo cargado correctamente: {filename}")

    # --- Preprocesar timestamps ---
    df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
    df["timestamp"] = df["timestamp"].dt.tz_localize(None)
    df["date_only"] = df["timestamp"].dt.date

    # --- Determinar fecha a actualizar: última fila (previa) ---
    last_date = df["date_only"].max()
    last_idx = df[df["date_only"] == last_date].index

    if len(last_idx) == 0:
        print("⚠️ No se encontró fila para actualizar, agregando nueva fila.")
        new_row = {
            "timestamp": (datetime.utcnow().date() - timedelta(days=1)).strftime("%Y-%m-%d"),
            "vader": vader_promedio,
            "finbert": finbert_promedio
        }
        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    else:
        # Actualizar valores de la última fila
        df.loc[last_idx, "vader"] = vader_promedio
        df.loc[last_idx, "finbert"] = finbert_promedio
        print(f"🟢 Actualizado sentiment para {last_date} en fila {last_idx[0]}")

    # --- Guardar ---
    df.drop(columns=["date_only"], inplace=True, errors="ignore")
    df.to_csv(filename, sep="\t", index=False)
    print(f"💾 Archivo actualizado y guardado: {filename} | Filas totales: {len(df)}")





# ===============================================
#           EJECUCIÓN PRINCIPAL
# ===============================================

for symbol, info in activos.items():
    descargar_noticias_y_calcular_sentiment(symbol, info["search_name"])

print("✅ Completed premarket (sentiment del día anterior actualizado)")
